In [2]:
import numpy as np
from sympy import *

Computing S-Polynomials

In [3]:
def spoly(f1,f2,order = 'lex'):
    LM1 = LM(f1,order = order)
    LT1 = LT(f1,order = order)
    LM2 = LM(f2,order = order)
    LT2 = LT(f2,order = order)
    numerator = lcm(LM1,LM2)
    S = numerator/LT1*f1 - numerator/LT2 * f2
    return(expand(S))

Example

In [4]:
x,y = var('x,y')
f1 = 3*(x**3)*y + 2*x*y - y**2
f2 = 2*x*(y**2)-5*y**3
print(spoly(f1,f2,order = 'lex'))

5*x**2*y**3/2 + 2*x*y**2/3 - y**3/3


Computing Groebner Basis using Buchberger's Algorithm

In [5]:
def GBasis_BuchAlg(F,order = 'lex'):
    Q = [g for g in F]
    while len(Q) > 0:
        g = Q.pop(0)
        temp = [f for f in F if not f == g]
        for f in temp:
            S = spoly(g,f, order = order)
            q,r = reduced(S, F, order = order)
            if not r == 0:
                F.append(r)
                Q.append(r)
    return(F)

Examples

In [6]:
x,y = var('x,y')
F = [x*y-x, x**2-y]
G = GBasis_BuchAlg(F, order = 'lex')
print(G)

[x*y - x, x**2 - y, y**2 - y]


In [7]:
x,y = var('x,y')
F = [2*x**2-4*x+y**2-4*y+3, x**2-2*x+3*y**2-12*y+9]
G = GBasis_BuchAlg(F, order = 'lex')
print(G)

[2*x**2 - 4*x + y**2 - 4*y + 3, x**2 - 2*x + 3*y**2 - 12*y + 9, -5*y**2/2 + 10*y - 15/2]


In [8]:
x,y = var('x,y')
F = [-x**2+x*y**3, x**3*y**2 - y]
G = GBasis_BuchAlg(F,order = 'grlex')
print(G)

[-x**2 + x*y**3, x**3*y**2 - y, -x**4 + y**2, -x*y + y**4]


Minimizing Groebner Basis

In [9]:
def minimize_GBasis(G, order = 'lex'):
    for i,g in enumerate(G):
        G[i] = g/LC(g,order = order)
    n = len(G)
    G2 = []
    for g in G:
        H = [f for f in G]
        H.remove(g)
        H = [LT(f,order = order) for f in H]
        LTg = LT(g,order = order)
        q,r = reduced(LTg, H, order = order)
        if not r == 0:
            G2.append(g)
    return(G2)
        
            

Examples

In [10]:
x,y = var('x,y')
F = [x**3 - 2*x*y, x**2*y-2*y**2 + x]
G = GBasis_BuchAlg(F, order = 'grlex')
MG = minimize_GBasis(G,order = 'grlex')
print(MG)

[x**2, -x/2 + y**2, x*y]


Reducing Groebner Basis

In [11]:
def reduce_GBasis(G, order = 'lex'):
    G = minimize_GBasis(G, order = order)
    for i,g in enumerate(G):
        H = [f for f in G if not f == g]
        q,r = reduced(g,H, order = order)
        G[i] = r
    return(G)

In [12]:
x,y = var('x,y')
F = [x**3 - 2*x*y, x**2*y-2*y**2 + x]
G = GBasis_BuchAlg(F, order = 'grlex')
RG = reduce_GBasis(G,order = 'grlex')
print(RG)
SG = groebner(F, order = 'grlex')
print(list(SG))

[x**2, -x/2 + y**2, x*y]
[x**2, x*y, -x/2 + y**2]


Putting it all together

In [13]:
def GrobAdd_Buchberger(G,f, order = 'lex'):
    G.append(f)
    Q = [f]
    while len(Q) > 0:
        g = Q.pop(0)
        temp = [f for f in G if not f == g]
        for f in temp: 
            S = spoly(g, f, order = order)
            q,r = reduced(S,G,order = order)
            if not r == 0: 
                G.append(r)
                Q.append(r)
    G = reduce_GBasis(G,order = order)
    return(G)

In [15]:
x,y = var('x,y')
F = [x**3 - 2*x*y, x**2*y-2*y**2 + x]
G = groebner(F,order = 'lex')
G = list(G)
print(G)
f = x**2 - x*y**2 + x**2*y**3
test = GrobAdd_Buchberger(G,f,order = 'lex')
print(test)
F.append(f)
test = groebner(F,order = 'lex')
print(list(test))

[x - 2*y**2, y**3]
[x - 2*y**2, y**3]
[x - 2*y**2, y**3]
